# 11 数据分析案例

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
# 一个cell输出多行语句
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 一、利用Python实现报表自动化
数据分析师 vs "表哥"

### 1.1 为什么要进行报表自动化
* 提高工作效率
* 减少错误

### 1.2 什么样的报表适合自动化
* 使用频率较高，如：日报、周报、月报
* 开发时间长短
* 需求变更频率高
* 流程标准

### 1.3 案例说明
假如你现在每天需要做一个表，这个表包括销售额、客流量、客单价这个三个指标的本月累计、上月同期、去年同期、环比、同比这几个指标

<img src='./image/case1_1.jpg' width='60%' />

做报表的源数据存放在一张订单表里面，包含了从去年至今的所有订单数据

<img src='./image/case1_2.jpg' width='60%' />

指标说明：
* 销售额 = 单价*销量
* 客流量 = 订单ID去重计数
* 客单价 = 销售额/客流量
* 本月 = 2018年2月
* 上月 = 2018年1月
* 去年同期 = 2017年2月

In [14]:
# parse_dates：将数据解析为时间格式
data = pd.read_csv('./data/order-14.1.csv', parse_dates=['成交时间'], encoding='gbk')
data.head()
data.info()

,商品ID,类别ID,门店编号,单价,销量,成交时间,订单ID
0,30006206.0,915000003.0,CDNL,25.23,0.328,2018-01-01,20170103CDLG000210052759
1,30163281.0,914010000.0,CDNL,2.00,2.000,2018-01-02,20170103CDLG000210052759
2,30200518.0,922000000.0,CDNL,19.62,0.230,2018-01-03,20170103CDLG000210052759
3,29989105.0,922000000.0,CDNL,2.80,2.044,2018-01-04,20170103CDLG000210052759
4,30179558.0,915000100.0,CDNL,47.41,0.226,2018-01-05,20170103CDLG000210052759


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3744 entries, 0 to 3743
Data columns (total 7 columns):
商品ID    3478 non-null float64
类别ID    3478 non-null float64
门店编号    3478 non-null object
单价      3478 non-null float64
销量      3478 non-null float64
成交时间    3478 non-null datetime64[ns]
订单ID    3478 non-null object
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 204.9+ KB


In [15]:
# 计算本月相关指标
# 本月数据
month_this = data[(data['成交时间'] >= datetime(2018, 2, 1)) & (data['成交时间'] <= datetime(2018, 2, 28))]
# 本月销售额
sales_this = (month_this['销量']*month_this['单价']).sum()
# 客流量
traffic_this = month_this['订单ID'].drop_duplicates().count()
# 客单价计算
s_t_this = sales_this/traffic_this
sales_this
traffic_this
s_t_this

10412.78007

343

30.357959387755105

In [16]:
# 计算上月相关指标
month_pre = data[(data['成交时间'] >= datetime(2018, 1, 1)) & (data['成交时间'] <= datetime(2018, 1, 31))]
sales_pre = (month_pre['销量']*month_pre['单价']).sum()
# 客流量
traffic_pre = month_pre['订单ID'].drop_duplicates().count()
# 客单价计算
s_t_pre = sales_pre/traffic_pre
sales_pre
traffic_pre
s_t_pre

9940.97291

315

31.55864415873016

In [20]:
def get_month_data(data):
    # 本月销售额
    sales = (data['销量']*data['单价']).sum()
    # 客流量
    traffic = data['订单ID'].drop_duplicates().count()
    # 客单价计算
    s_t = sales/traffic
    return (sales, traffic, s_t)

month_last_year = data[(data['成交时间'] >= datetime(2017, 2, 1)) & (data['成交时间'] <= datetime(2017, 2, 28))]
sales_this, traffic_this, s_t_this = get_month_data(month_this)
sales_pre, traffic_pre, s_t_pre = get_month_data(month_pre)
sales_last_year, traffic_last_year, s_t_last_year = get_month_data(month_last_year)

report = pd.DataFrame([[sales_this, sales_pre, sales_last_year], 
                       [traffic_pre, traffic_pre, traffic_last_year], 
                       [s_t_pre, s_t_pre, s_t_last_year]],
                     columns=['本月累计', '上月同期', '去年同期'],
                     index=['销售额', '客流量', '客单价'])
report

,本月累计,上月同期,去年同期
销售额,10412.780070,9940.972910,8596.313470
客流量,315.000000,315.000000,262.000000
客单价,31.558644,31.558644,32.810357


In [21]:
# 添加同比与环比
report['环比'] = report['本月累计'] / report['上月同期'] - 1
report['同比'] = report['本月累计'] / report['去年同期'] - 1
report

,本月累计,上月同期,去年同期,环比,同比
销售额,10412.780070,9940.972910,8596.313470,0.047461,0.211308
客流量,315.000000,315.000000,262.000000,0.000000,0.202290
客单价,31.558644,31.558644,32.810357,0.000000,-0.038150


In [22]:
# 导出结果
report.to_excel('./data/order-report.xlsx', encoding='utf-8')

## 二、自动发送邮件
报表做出来后要发送给老板看，每天自动发送邮件

In [25]:
import smtplib
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr, formataddr
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

# 发件人邮箱
asender = 'liupan6888@163.com'
# 收件人邮箱
areceiver = 'liupan6888@163.com'
# 抄送人邮箱
acc = 'liupan6888@163.com'
# 邮件主题
asubject = '这是一份测试邮件'

# 发件人地址
from_addr = 'liupan6888@163.com'
# 密码
password = 'silian1878'

# 邮件设置
msg = MIMEMultipart()
msg['Subject'] = asubject
msg['to'] = areceiver
msg['Cc'] = acc
msg['from'] = '张三'

# 正文
body = '您好，这是一份测试邮件'

# 添加邮件正文
msg.attach(MIMEText(body, 'plain', 'utf-8'))

# 添加附件
xlsxpart = MIMEApplication(open('./data/order-report.xlsx', 'rb').read())
xlsxpart.add_header('Content-Disposition', 'attachment', filename='订单销售分析报告.xlsx')
msg.attach(xlsxpart)

# 设置邮箱服务器地址及端口
smtp_server = 'smtp.163.com'
server = smtplib.SMTP(smtp_server, 25)
server.set_debuglevel(1)
# 登录邮箱
server.login(from_addr, password)
# 发送邮件
server.sendmail(from_addr, areceiver.split(',')+acc.split(','),
               msg.as_string())

server.quit()

send: 'ehlo pandeMacBook-Pro.local\r\n'
reply: b'250-mail\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH LOGIN PLAIN\r\n'
reply: b'250-AUTH=LOGIN PLAIN\r\n'
reply: b'250-coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UrZjTVkUCa0xDrUUUUj\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'mail\nPIPELINING\nAUTH LOGIN PLAIN\nAUTH=LOGIN PLAIN\ncoremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UrZjTVkUCa0xDrUUUUj\nSTARTTLS\n8BITMIME'
send: 'AUTH PLAIN AGxpdXBhbjY4ODhAMTYzLmNvbQBzaWxpYW4xODc4\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'


(235, b'Authentication successful')

send: 'mail FROM:<liupan6888@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'rcpt TO:<liupan6888@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'rcpt TO:<liupan6888@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Content-Type: multipart/mixed; boundary="===============0674540787439486341=="\r\nMIME-Version: 1.0\r\nSubject: =?utf-8?b?6L+Z5piv5LiA5Lu95rWL6K+V6YKu5Lu2?=\r\nto: liupan6888@163.com\r\nCc: liupan6888@163.com\r\nfrom: =?utf-8?b?5byg5LiJ?=\r\n\r\n--===============0674540787439486341==\r\nContent-Type: text/plain; charset="utf-8"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: base64\r\n\r\n5oKo5aW977yM6L+Z5piv5LiA5Lu95rWL6K+V6YKu5Lu2\r\n\r\n--===============0674540787439486341==\r\nContent-T

{}

send: 'quit\r\n'
reply: b'221 Bye\r\n'
reply: retcode (221); Msg: b'Bye'


(221, b'Bye')